<a href="https://colab.research.google.com/github/xinnazim/esaa/blob/main/%ED%95%AD%EA%B3%B5%ED%8E%B8_%EC%A7%80%EC%97%B0_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 데이터 및 라이브러리 로딩**

In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') #matplotlib 한글 깨짐 방지

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import log_loss, accuracy_score, f1_score

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 데이터 불러오기


train=pd.read_csv("/content/drive/MyDrive/air/train.csv")
test = pd.read_csv("/content/drive/MyDrive/air/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/air/sample_submission.csv") # 제출용 파일

# **1. EDA**

In [4]:
train.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN


In [5]:
train.tail()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN
999999,TRAIN_999999,3,26,1800.0,2056.0,0,0,RSW,14635,Florida,DTW,11433,Michigan,1084.0,NaN,DL,19790.0,N695DL,NaN


In [6]:
test.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,TEST_000000,12,16,1156.0,NaN,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,NaN,N79402
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,NaN,746.0,Delta Air Lines Inc.,DL,19790.0,N3765
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,LGA,12953,New York,733.0,United Air Lines Inc.,UA,19977.0,N413UA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,LAX,12892,California,337.0,Southwest Airlines Co.,WN,19393.0,N905WN
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,LAX,12892,California,2343.0,JetBlue Airways,B6,20409.0,N945JT


In [7]:
submission.head()

,ID,Not_Delayed,Delayed
0,TEST_000000,0,1
1,TEST_000001,0,1
2,TEST_000002,0,1
3,TEST_000003,0,1
4,TEST_000004,0,1


In [8]:
print(train.shape)
print(test.shape)
print(submission.shape)

(1000000, 19)
(1000000, 18)
(1000000, 3)


* ID : 샘플 고유 id
* Month: 해당 항공편의 출발 월
* Day_of_Month: Month에 해당하는 월의 날짜
* Estimated_Departure_Time: 전산 시스템을 바탕으로 측정된 비행기의 출발 시간 (현지 시각, HH:MM 형식)
* Estimated_Arrival_Time: 전산 시스템을 바탕으로 측정된 비행기의 도착 시간 (현지 시각, HH:MM 형식)
* Cancelled: 해당 항공편의 취소 여부 (0: 취소되지 않음, 1: 취소됨)
* Diverted: 해당 항공편의 경유 여부 (0: 취소되지 않음, 1: 취소됨)
* Origin_Airport: 해당 항공편 출발 공항의 고유 코드 (IATA 공항 코드)
* Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
* Origin_State: 해당 항공편 출발 공항이 위치한 주의 이름
* Destination_Airport: 해당 항공편 도착 공항의 고유 코드 (IATA 공항 코드)
* Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
* Destination_State: 해당 항공편 도착 공항이 위치한 주의 이름
* Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
* Airline: 해당 항공편을 운항하는 항공사
* Carrier_Code(IATA): 해당 항공편을 운항하는 항공사의 고유 코드
(IATA 공항 코드, 단 다른 항공사가 같은 코드를 보유할 수도 있음)
* Carrier_ID(DOT): 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
* Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호
* Delay: 항공편 지연 여부 (Not_Delayed, Delayed)

In [9]:
train.info()
# null: Estimated_Departure_Time, Estimated_Arrival_Time, Origin_State ,Destination_State, Airline, Carrier_Code(IATA), Carrier_ID(DOT)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

In [ ]:
pd.options.display.float_format = '{:.1f}'.format # 소숫점 자리수 출력값 format 설정
train.describe()

import numpy as np
from google.colab import autoviz
df_1111038813399659526 = autoviz.get_registered_df('df_1111038813399659526')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_1111038813399659526, *['Month'], **{})
chart

import numpy as np
from google.colab import autoviz
df_4172685511932574644 = autoviz.get_registered_df('df_4172685511932574644')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_4172685511932574644, *['Day_of_Month'], **{})
chart

import numpy as np
from google.colab import autoviz
df_2126788988919502770 = autoviz.get_registered_df('df_2126788988919502770')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_2126788988919502770, *['Cancelled'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5528071244011115416 = autoviz.get_registered_df('df_5528071244011115416')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_5528071244011115416, *['Diverted'], **{})
chart

import numpy as np
from google.colab import autoviz
df_902509617303174522 = autoviz.get_registered_df('df_902509617303174522')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_902509617303174522, *['Month'], **{})
chart

...

import numpy as np
from google.colab import autoviz
df_8512564273322916218 = autoviz.get_registered_df('df_8512564273322916218')

def scatter_plots(df, colname_pairs, scatter_plot_size=2.5, size=8, alpha=.6):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * scatter_plot_size, scatter_plot_size))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    ax.scatter(df[x_colname], df[y_colname], s=size, alpha=alpha)
    plt.xlabel(x_colname)
    plt.ylabel(y_colname)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(df_8512564273322916218, *[[['Month', 'Day_of_Month'], ['Day_of_Month', 'Cancelled'], ['Cancelled', 'Diverted'], ['Diverted', 'Origin_Airport_ID']]], **{})
chart

import numpy as np
from google.colab import autoviz
df_544560340235665106 = autoviz.get_registered_df('df_544560340235665106')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_544560340235665106, *['Estimated_Departure_Time', 'Month', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_4420138365797178793 = autoviz.get_registered_df('df_4420138365797178793')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_4420138365797178793, *['Estimated_Departure_Time', 'Day_of_Month', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_3850973570545106525 = autoviz.get_registered_df('df_3850973570545106525')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_3850973570545106525, *['Estimated_Departure_Time', 'Cancelled', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_3624218969784947305 = autoviz.get_registered_df('df_3624218969784947305')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_3624218969784947305, *['Estimated_Departure_Time', 'Diverted', None], **{})
chart

In [11]:
train.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

In [12]:
test.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                106505
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           106523
Distance                         0
Airline                     106527
Carrier_Code(IATA)          108993
Carrier_ID(DOT)             109006
Tail_Number                      0
dtype: int64

* iD : 샘플 고유 id
* Month: 해당 항공편의 출발 월(날씨에 영향을 받을거라 생각)
* Estimated_Departure_Time: null o, 비행기의 출발 시간 (현지 시각, HH:MM 형식)
* Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID) 코드 대신 ID 사용
* Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
* Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
* Airline: null o, 해당 항공편을 운항하는 항공사
* Carrier_ID(DOT): null o, 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
* Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호
* Delay: 항공편 지연 여부 (Not_Delayed, Delayed)

# **2. 데이터 전처리**

* airline과 carrier_ID가 대응되어서 결측값을 채울 수 있음

In [4]:
# airline, carrier_ID 결측치 채우기
cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = ~train['Airline'].isnull()
print("Carrier_ID(DOT) 복구 가능한 데이터의 개수 :", len(train.loc[cond1 & cond2, :]))

Carrier_ID(DOT) 복구 가능한 데이터의 개수 : 97114


In [5]:
# airline to carrier id, dictinary 만들기
# 모두 데이터가 존재하는 열에서 Dict[Airline] = carrier_ID(DOT) 가 되도록 dictionary 생성
airline_to_cid = {}
for _, row in train[(~train['Carrier_ID(DOT)'].isnull() & ~train['Airline'].isnull())].iterrows():
    airline_to_cid[row['Airline']] = row['Carrier_ID(DOT)']

In [6]:
# carrier_ID 복구하기
def to_cid(x):
    return airline_to_cid[x]

cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = ~train['Airline'].isnull()
train.loc[cond1&cond2, 'Carrier_ID(DOT)'] = train.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))

In [7]:
# 복구 안 된 Carrier_ID 제거
train = train.dropna(subset=['Carrier_ID(DOT)'], how='any', axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988117 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        988117 non-null  object 
 1   Month                     988117 non-null  int64  
 2   Day_of_Month              988117 non-null  int64  
 3   Estimated_Departure_Time  880391 non-null  float64
 4   Estimated_Arrival_Time    880328 non-null  float64
 5   Cancelled                 988117 non-null  int64  
 6   Diverted                  988117 non-null  int64  
 7   Origin_Airport            988117 non-null  object 
 8   Origin_Airport_ID         988117 non-null  int64  
 9   Origin_State              880378 non-null  object 
 10  Destination_Airport       988117 non-null  object 
 11  Destination_Airport_ID    988117 non-null  int64  
 12  Destination_State         880335 non-null  object 
 13  Distance                  988117 non-null  f

In [8]:
# test는 복구가 안된 row도 예측에 필요하기 때문에 airline, carrier_ID가 모두 없는 row는 최빈값으로 채움
# Airline, Carrier_Code 둘 다 없으면 최빈 값으로 대체
NaN_col = ['Carrier_ID(DOT)']
cond1 = test['Airline'].isnull()
cond2 = test['Carrier_ID(DOT)'].isnull()

for col in NaN_col:
    mode = test[col].mode()[0]
    test.loc[cond1&cond2, col] = mode

# 나머진 Airline에서 대체
cond1 = test['Carrier_ID(DOT)'].isnull()
cond2 = ~test['Airline'].isnull()
test.loc[cond1&cond2, 'Carrier_ID(DOT)'] = test.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))


In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  891016 non-null   float64
 4   Estimated_Arrival_Time    890952 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              893495 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         893477 non-null   object 
 13  Distance                  10

In [9]:
# 필요없는 columns 제거

col_drop = ['Day_of_Month', 'Cancelled', 'Diverted', 'Origin_Airport', 'Destination_Airport', 'Carrier_Code(IATA)', 'Airline', 'Origin_State', 'Destination_State']
train = train.drop(col_drop, axis=1)
test = test.drop(col_drop, axis=1)

In [23]:
train.isna().sum()

ID                               0
Month                            0
Estimated_Departure_Time    107726
Estimated_Arrival_Time      107789
Origin_Airport_ID                0
Destination_Airport_ID           0
Distance                         0
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       736101
dtype: int64

In [22]:
train.head()

,ID,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,NaN,NaN,13851,12191,419.0,19393.0,N7858A,NaN
1,TRAIN_000001,8,740.0,1024.0,13930,14869,1250.0,20304.0,N125SY,NaN
2,TRAIN_000002,9,1610.0,1805.0,11057,12953,544.0,19805.0,N103US,NaN
3,TRAIN_000003,7,905.0,1735.0,12892,11618,2454.0,19977.0,N595UA,NaN
4,TRAIN_000004,1,900.0,1019.0,14771,10157,250.0,20304.0,N161SY,NaN


In [10]:
# 출발,도착 시간 결측값 처리
# 결측값을 최빈값으로 대체함
NaN_col = ['Estimated_Departure_Time','Estimated_Arrival_Time']

for col in NaN_col:
  mode=train[col].mode()[0]
  train[col]=train[col].fillna(mode)

  if col in test.columns:
    test[col]=test[col].fillna(mode)

In [11]:
train.head()

,ID,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,600.0,1900.0,13851,12191,419.0,19393.0,N7858A,NaN
1,TRAIN_000001,8,740.0,1024.0,13930,14869,1250.0,20304.0,N125SY,NaN
2,TRAIN_000002,9,1610.0,1805.0,11057,12953,544.0,19805.0,N103US,NaN
3,TRAIN_000003,7,905.0,1735.0,12892,11618,2454.0,19977.0,N595UA,NaN
4,TRAIN_000004,1,900.0,1019.0,14771,10157,250.0,20304.0,N161SY,NaN


In [11]:
# 질적 변수 수치화

qual_col = ['Origin_Airport_ID', 'Destination_Airport_ID',  'Carrier_ID(DOT)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])

    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])



In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988117 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        988117 non-null  object 
 1   Month                     988117 non-null  int64  
 2   Estimated_Departure_Time  988117 non-null  float64
 3   Estimated_Arrival_Time    988117 non-null  float64
 4   Origin_Airport_ID         988117 non-null  int64  
 5   Destination_Airport_ID    988117 non-null  int64  
 6   Distance                  988117 non-null  float64
 7   Carrier_ID(DOT)           988117 non-null  float64
 8   Tail_Number               988117 non-null  object 
 9   Delay                     252016 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 82.9+ MB


In [12]:
#레이블 없는 데이터 제거
# train = train.dropna()
nolabel = train[train['Delay'].isnull()]
train = train.dropna()
nolabel

,ID,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,600.0,1900.0,253,159,419.0,0,4318,NaN
1,TRAIN_000001,8,740.0,1024.0,257,332,1250.0,12,310,NaN
2,TRAIN_000002,9,1610.0,1805.0,74,205,544.0,4,140,NaN
3,TRAIN_000003,7,905.0,1735.0,196,119,2454.0,6,3020,NaN
4,TRAIN_000004,1,900.0,1019.0,323,7,250.0,12,555,NaN
...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,936.0,1243.0,257,271,678.0,6,2476,NaN
999996,TRAIN_999996,5,920.0,1028.0,122,243,223.0,12,2293,NaN
999997,TRAIN_999997,6,800.0,1340.0,249,159,1642.0,0,993,NaN
999998,TRAIN_999998,9,1613.0,1824.0,45,22,214.0,3,6205,NaN


In [15]:
train = train.replace({'Not_Delayed':0, 'Delayed':1})

In [18]:
train_x = train.drop(columns=['ID', 'Delay'])
train_y = train['Delay']
test_x = test.drop(columns=['ID'])